### Import libraries

In [3]:
import PyPDF2 
import textract
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk.data
import nltk
import glob
#from glob import glob
import os
import re
import string

In [9]:
#nltk.download('stopwords')
#nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kathrynkundrod/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/kathrynkundrod/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Capture text from pdf files, a singular example
https://medium.com/@rqaiserr/how-to-convert-pdfs-into-searchable-key-words-with-python-85aab86c544f

In [12]:

filename = "Austin_Transcripts/scrape2/2015_01_29.pdf"

pdfFileObj = open(filename,'rb')

#The pdfReader variable is a readable object that will be parsed
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

#discerning the number of pages will allow us to parse through all #the pages
num_pages = pdfReader.numPages
count = 0
text = ""

#The while loop will read each page
while count < num_pages:
    pageObj = pdfReader.getPage(count)
    count +=1
    text += pageObj.extractText()

#print(text)

## Capture text from pdf files, loop through all files

In [15]:
# 'glob' files, i.e., make a list of all with the same desired attribute
# in this case, all pdfs in the folder Austin_Transcripts
filelist = glob.glob("Austin_Transcripts/*.pdf")
#filelist

In [16]:
fileroot, fileext = os.path.splitext(pdffile)
output_filename = fileroot+'.txt'

In [17]:
# for each file in filelist...
for pdffile in filelist:
    
    # Open the pdf file and make it a pdf object
    pdfFileObj = open(pdffile,'rb')

    # The pdfReader variable is a readable object that will be parsed
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

    # Discerning the number of pages will allow us to parse through all #the pages
    num_pages = pdfReader.numPages
    count = 0
    text = ""

    # This while loop will read each page
    # The 'text' output is what we want to save for each pdf
    while count < num_pages:
        pageObj = pdfReader.getPage(count)
        count +=1
        text += pageObj.extractText()

    # Create an output file and put "text" in there
    file = open(output_filename, "w")
    file.write(text)
    fileroot, fileext = os.path.splitext(pdffile)
    output_filename = fileroot+'.txt'
    file.close()   


## Cleaning text files, a singular example

In [67]:
# name the file before opening so it's easier to iterate on later
text_ex =  "Austin_Transcripts/2017_03_23.txt"
f = open(text_ex, 'r')

# turns f file into a string
content = f.read()
#type(content)   # it's a string
#print(content)  # see, it's a string

# always close files after pulling out the "content" or whatever you want to avoid overwriting
f.close()

In [68]:
## Basic text cleanup

# remove time stamp
# this re command removes everything between "[]"
text_cleaning = re.sub("[\[].*?[\]]", "", content)

# remove the new line instances
text_cleaning = text_cleaning.replace("\n", " ")

# subbing repeated white space for a single space
text_cleaning = re.sub('\s\s+', ' ', text_cleaning)

# remove header
text_cleaning = re.sub(r'.*====', '=', text_cleaning)

# remove all punctuation that is not ">" or ":"
# we need these puncts to parse speakers
remove = string.punctuation
remove = remove.replace(">", "") # don't remove carats
remove = remove.replace(":", "") # keep these too, which helps me see when CC members are speaking
pattern = r"[{}]".format(remove) # create the pattern
text_cleaning = re.sub(pattern, "", text_cleaning) 

## Split text by speaker. Each ">>" indicates a new speaker
# split by speakers - now I have a list of string, each string a new speaker
speaker_strings = text_cleaning.split(">>")
speaker_strings[10:20]
#type(speaker_strings)

## Now, remove strings said by a CCM or Mayor, i.e., those tagged with a name  
# initializing substring
# want to recognize [any alpha character + :]
subs = '\w+:' # "\w" indicates any alpha character 
  
# using list comprehension to return strings without the substring above
# I want to print all lists that DO NOT contain the substring
#speaker_strings2 = [i for i in speaker_strings1 if not subs in i] 
speaker_strings = [string for string in speaker_strings if not re.search(subs, string)]

# now, same thing to remove the prayer
subs2 = "Amen"
speaker_strings = [string for string in speaker_strings if not re.search(subs2, string)]

speaker_strings_long = []
for list_element in speaker_strings:
    if len(list_element) >= 100:
        speaker_strings_long.append(list_element)
else:
    pass

# concatenate into a string for eventual export
speaker_strings_long = '_'.join(speaker_strings_long)

## Now, loop through all files to clean and parse by speaker

In [57]:
# making a list of all txt files, to run through loop later
txt_list = glob.glob("Austin_Transcripts/*.txt")
#txt_list

In [59]:
fileroot, fileext = os.path.splitext(raw_text)
output_filename = fileroot+'clean_.txt'

In [ ]:
for raw_text in txt_list:
    
    # basic text cleaning
    text_cleaning = re.sub("[\[].*?[\]]", "", content)
    text_cleaning = text_cleaning.replace("\n", " ")
    text_cleaning = re.sub('\s\s+', ' ', text_cleaning)
    text_cleaning = re.sub(r'.*====', '=', text_cleaning)
    remove = string.punctuation
    remove = remove.replace(">", "") # don't remove carats
    remove = remove.replace(":", "") # keep these too, which helps me see when CC members are speaking
    pattern = r"[{}]".format(remove) # create the pattern
    text_cleaning = re.sub(pattern, "", text_cleaning) 

    # separate speakers
    speaker_strings = text_cleaning.split(">>")
    
    ## Now, remove strings said by a CCM or Mayor, i.e., those tagged with a name  
    # initializing substring
    # want to recognize [any alpha character + :]
    subs = '\w+:' # "\w" indicates any alpha character 

    # using list comprehension to return strings without the substring above
    # I want to print all lists that DO NOT contain the substring
    #speaker_strings2 = [i for i in speaker_strings1 if not subs in i] 
    speaker_strings = [string for string in speaker_strings if not re.search(subs, string)]

    # now, same thing to remove the prayer
    subs2 = "Amen"
    speaker_strings = [string for string in speaker_strings if not re.search(subs2, string)]

    speaker_strings_long = []
    for list_element in speaker_strings:
        if len(list_element) >= 100:
            speaker_strings_long.append(list_element)
    else:
        pass
    
    # Put it in a string for export
    speaker_strings_long = '_'.join(speaker_strings_long)

    # Create an output file and put "text" in there
    file = open(output_filename, "w")
    file.write(speaker_strings_long)
    fileroot, fileext = os.path.splitext(raw_text)
    output_filename = fileroot+'_clean.txt'
    file.close()   


## Parsing text files by individual, loop through all files

## Basic keyword maker

In [6]:
#The word_tokenize() function will break our text phrases into individual words
text_ex = 
tokens = word_tokenize(text_ex)

#we'll create a new list which contains punctuation we wish to clean
punctuations = ['(',')',';',':','[',']',',']

#We initialize the stopwords variable which is a list of words like #"The", "I", "and", etc. that don't hold much value as keywords
stop_words = stopwords.words('english')

#We create a list comprehension which only returns a list of words #that are NOT IN stop_words and NOT IN punctuations.
#keywords = [word for word in tokens if not word in stop_words and not word in punctuations]
keywords = [word for word in tokens if not word in stop_words and not word in punctuations]

In [18]:
#print(keywords)